In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Sail2304/LoanTap.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sail2304"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b9b3f722c5b2cfcfba8e769fcd6c4ffd37e6136b"

In [2]:
%pwd

'd:\\LoanTap\\LoanTap\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\LoanTap\\LoanTap'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    scaler_path: Path
    ohencoder_path: Path
    le_grade_path: Path
    le_subgrade_path: Path
    le_emp_length_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
from src.LoanTapPred.constants import *
from src.LoanTapPred.utils.common import read_yaml, create_directories,save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            scaler_path=config.scaler_path,
            ohencoder_path=config.ohencoder_path,
            le_grade_path=config.le_grade_path,
            le_subgrade_path=config.le_subgrade_path,
            le_emp_length_path=config.le_emp_length_path,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri=config.mlflow_uri
        )

        return model_evaluation_config

In [13]:
import os 
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [14]:
class ModelEvaluation():
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual,pred)
        recall = recall_score(actual,pred)
        f1 = f1_score(actual, pred)
        cm = confusion_matrix(actual,pred)

        return accuracy, precision, recall, f1, cm
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        sc = joblib.load(self.config.scaler_path)

        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[[self.config.target_column]]
        X_test_scaled=sc.transform(X_test)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            pred = model.predict(X_test_scaled)
            (accuracy, precision, recall, f1, cm) = self.eval_metrics(y_test, pred)
            TN, FP, FN, TP = cm.ravel()
            # saving metrics as local
            scores = {"accuracy": accuracy, 
                      "precision":precision, 
                      "recall": recall, 
                      "f1": f1,
                      "TN": int(TN),
                      "FP": int(FP),
                      "FN": int(FN),
                      "TP": int(TP)
            }

            
            save_json(path=Path(self.config.metric_file_name), data=scores)


            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            mlflow.log_metric("TN", TN)
            mlflow.log_metric("FP", FP)
            mlflow.log_metric("FN", FN)
            mlflow.log_metric("TP", TP)
            mlflow.log_artifact(self.config.scaler_path)
            mlflow.log_artifact(self.config.ohencoder_path)
            mlflow.log_artifact(self.config.le_grade_path)
            mlflow.log_artifact(self.config.le_subgrade_path)
            mlflow.log_artifact(self.config.le_emp_length_path)

            #model registery does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="LoanTapLR")
            else:
                mlflow.sklearn.log_model(model, "model")
    



            




In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-23 12:55:58,023: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 12:55:58,025: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-23 12:55:58,028: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-23 12:55:58,028: INFO: common: created directory at: artifacts]
[2025-01-23 12:55:58,028: INFO: common: created directory at: artifacts/model_evaluation]


[2025-01-23 12:55:58,609: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/01/23 12:56:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LoanTapLR'.
2025/01/23 12:56:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LoanTapLR, version 1
Created version '1' of model 'LoanTapLR'.


🏃 View run wise-worm-35 at: https://dagshub.com/Sail2304/LoanTap.mlflow/#/experiments/0/runs/aba6d25108a3480fbab90252a6f19e8d
🧪 View experiment at: https://dagshub.com/Sail2304/LoanTap.mlflow/#/experiments/0
